In [3]:
import tensorflow as tf
import pandas as pd

from models.FCN import FCN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *

df = read_dataset("ArrowHead")

X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)
# apply one-hot encoder
y_train=to_categorical(df[1])
y_test=to_categorical(df[3])

model = FCN(input_size=X_train.shape[1:] , nb_classes=y_train.shape[1], filters=[128,256,128])
model.compile(optimizer=Adam(learning_rate=0.01), loss=CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

test (36, 251, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 251, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 251, 128)          1152      
                                                                 
 batch_normalization (Batch  (None, 251, 128)          512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 251, 128)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 251, 256)          164096    
                                                                 
 batch_normalization_1 (Bat  (None, 251, 256)          1024      
 chNormalization)                          

2024-02-16 15:31:46.514831: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 15:31:46.552418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 15:31:46.552685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
import tensorflow as tf
import pandas as pd

from models.FCN import FCN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *



results=[]
histories=[]

try:
    for dataset in  UNIVARIATE_DATASET_NAMES_2018:
        with tf.device("/device:GPU:0"):
            # Load data
            df = read_dataset(dataset)

            X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
            X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)

            # apply one-hot encoder
            y_train=to_categorical(df[1])
            y_test=to_categorical(df[3])
            print("test",X_train.shape)

            # Compile the model
            model = FCN(input_size=X_train.shape[1:] , nb_classes=y_train.shape[1], filters=[128,256,128])
            model.compile(optimizer=Adam(learning_rate=0.01), loss=CategoricalCrossentropy(), metrics=['accuracy'])

            callbacks =[
                tf.keras.callbacks.ModelCheckpoint(
                    f"best_models/best_model_distiller_{dataset}.tf", save_weights_only=True, monitor="lr"
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="lr", factor=0.5, patience=50, min_lr=0.0001
                ),
            ]

            # Train the model
            history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data= (df[2], y_test), callbacks = callbacks)

            # Evaluate the model
            train_loss, train_accuracy = model.evaluate(X_train, y_train)
            test_loss, test_accuracy = model.evaluate(X_test, y_test)

            results.append([dataset, train_loss, train_accuracy, test_loss, test_accuracy])
            histories.append(history.history)
        # clear_gpu_memory()
            
    pd.DataFrame(results, columns=["Dataset","Train loss", "Train accuracy","Test loss", "Test accuracy"]).to_csv("result.csv", index=False)

except Exception as e:
    print("An error occurred:", e)


for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i])

test (36, 251, 1)
salam
Epoch 1/1000


2024-02-16 15:33:53.294738: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-16 15:33:54.629141: I external/local_xla/xla/service/service.cc:168] XLA service 0x560a7a872cc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-16 15:33:54.629163: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P400, Compute Capability 6.1
2024-02-16 15:33:54.634276: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708094034.710756  112611 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 [==============================] - 7s 1s/step - loss: 1.2106 - accuracy: 0.3056 - val_loss: 2.0402 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - 0s 125ms/step - loss: 1.0299 - accuracy: 0.4722 - val_loss: 3.2486 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 3/1000
3/3 [==============================] - 0s 135ms/step - loss: 0.9827 - accuracy: 0.5556 - val_loss: 6.1188 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 4/1000
3/3 [==============================] - 0s 120ms/step - loss: 0.9651 - accuracy: 0.5278 - val_loss: 9.4277 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 5/1000
3/3 [==============================] - 0s 126ms/step - loss: 0.9694 - accuracy: 0.5556 - val_loss: 10.9245 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 6/1000
3/3 [==============================] - 0s 116ms/step - loss: 0.9513 - accuracy: 0.5556 - val_loss: 8.7621 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 7/1000
3/3 [==============================] - 0s 116ms/step - loss: 0.9097 - ac

In [8]:
results = pd.read_csv("result.csv")
results

,Dataset,Train loss,Train accuracy,Test loss,Test accuracy
0,ArrowHead,0.678126,0.638889,3.620084,0.485714
1,Wine,0.691872,0.526316,0.695424,0.500000
2,OliveOil,1.282252,0.433333,1.299150,0.400000
3,Car,1.151810,0.400000,5.328006,0.333333
4,BeetleFly,0.000000,1.000000,46007.804688,0.850000
5,Yoga,0.689401,0.543333,0.690652,0.535667
6,InlineSkate,1.921015,0.180000,1.924656,0.154545
7,FaceAll,2.639132,0.071429,2.639947,0.039053
8,Ham,0.685378,0.532110,0.693511,0.514286
9,MoteStrain,0.000000,1.000000,1332.437378,0.859425


In [4]:
results = pd.read_csv("result.csv")
results

,Dataset,Train loss,Train accuracy,Test loss,Test accuracy
0,ArrowHead,0.003899,1.000000,7.387911,0.685714
1,Wine,0.691779,0.526316,0.694387,0.500000
2,OliveOil,1.282821,0.433333,1.296988,0.400000
3,Car,0.352149,0.833333,1.172970,0.616667
4,BeetleFly,0.000000,1.000000,1421.919678,0.800000
5,Yoga,0.689453,0.543333,0.690578,0.535667
6,InlineSkate,1.920965,0.180000,4.587185,0.150909
7,FaceAll,2.629645,0.075000,2.915818,0.082249
8,Ham,0.000000,1.000000,8.105940,0.733333
9,MoteStrain,0.000000,1.000000,2.603525,0.861022


In [6]:
results_uea=pd.read_csv("results-uea-avg-std.csv")
results_uea

,Unnamed: 0,MLP,FCN,ResNet,Encoder,MCNN,t-LeNet,MCDCNN,Time-CNN,TWIESN
0,50words,68.4(7.1),62.7(6.1),74.0(1.5),72.3(1.0),22.0(24.3),12.5(0.0),58.9(5.3),62.1(1.0),49.6(2.6)
1,Adiac,39.7(1.9),84.4(0.7),82.9(0.6),48.4(2.5),2.2(0.6),2.0(0.0),61.0(8.7),37.9(2.0),41.6(4.5)
2,ArrowHead,77.8(1.2),84.3(1.5),84.5(1.2),80.4(2.9),33.9(4.7),30.3(0.0),68.5(6.7),72.3(2.6),65.9(9.4)
3,Beef,72.0(2.8),69.7(4.0),75.3(4.2),64.3(5.0),20.0(0.0),20.0(0.0),56.3(7.8),76.3(1.1),53.7(14.9)
4,BeetleFly,87.0(2.6),86.0(9.7),85.0(2.4),74.5(7.6),50.0(0.0),50.0(0.0),58.0(9.2),89.0(3.2),73.0(7.9)
...,...,...,...,...,...,...,...,...,...,...
82,uWaveGestureLibrary_Z,69.7(0.2),72.6(0.5),75.0(0.4),71.1(0.5),18.0(18.4),12.1(0.0),65.0(1.8),64.2(0.9),56.5(2.0)
83,wafer,99.6(0.0),99.7(0.0),99.9(0.1),99.6(0.0),91.3(4.4),89.2(0.0),99.2(0.3),96.1(0.1),91.4(0.5)
84,yoga,85.5(0.4),83.9(0.7),87.0(0.9),82.0(0.6),53.6(0.0),53.6(0.0),76.2(3.9),78.1(0.7),60.7(1.9)
85,Average_Rank,4.611765,2.682353,1.994118,3.682353,8.017647,8.417647,5.376471,4.970588,5.247059


In [9]:
df = pd.read_csv("results-uea-avg-std.csv")

# remove std
def remove_parenthesis(value):
    return float(value.split("(")[0])

def convert_to_percent(value):
    return float("%.2f" % float(value*100))

df.iloc[:, 1:] = df.iloc[:, 1:].map(remove_parenthesis)
results["Train accuracy"]=results["Train accuracy"].apply(convert_to_percent)
results["Test accuracy"]=results["Test accuracy"].apply(convert_to_percent)

df['Unnamed: 0'] = df['Unnamed: 0'].str.lower()
dataset_names = pd.Series(UNIVARIATE_DATASET_NAMES_2018, name="Unnamed: 0").str.lower()
df_copy = df.merge(dataset_names, on='Unnamed: 0', how='right')
df_copy.dropna(axis=0, inplace=True)

results['Unnamed: 0'] = results['Dataset'].str.lower()

# merge and remove unused columns
result = results.merge(df_copy[["Unnamed: 0","MLP"]],on="Unnamed: 0",how="right").drop("Unnamed: 0",axis=1)
result

,Dataset,Train loss,Train accuracy,Test loss,Test accuracy,MLP
0,ArrowHead,1.369213e-05,100.00,5.680031,73.71,77.8
1,Wine,6.917889e-01,52.63,0.694254,50.00,56.5
2,OliveOil,1.282214e+00,43.33,1.298481,40.00,66.7
3,Car,1.869125e-01,93.33,1.410494,63.33,76.7
4,BeetleFly,0.000000e+00,100.00,4.377611,80.00,87.0
5,Yoga,6.893885e-01,54.33,0.690694,53.57,85.5
6,InlineSkate,1.742696e+00,26.00,3.922623,20.36,33.7
7,FaceAll,2.610312e+00,8.21,2.632383,4.02,79.3
8,Ham,1.421762e-08,100.00,7.896139,64.76,69.1
9,MoteStrain,0.000000e+00,100.00,3.974996,81.23,85.8


In [11]:
plot_1v1_perf(result,"MLP","Test accuracy")